In [2]:
import pandas as pd
import numpy as np

In [3]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [4]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [5]:
credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [6]:
movies=movies.merge(credits,left_on='id',right_on='movie_id',how='left')
popular_movies_df=movies[['id','title_x','popularity']]
movies=movies[['id','genres','keywords','overview','runtime','release_date','title_x','cast','crew']]

In [7]:
popular_movies_df=popular_movies_df.sort_values('popularity',ascending=False)
popular_movies_df.head(3)

,id,title_x,popularity
546,211672,Minions,875.581305
95,157336,Interstellar,724.247784
788,293660,Deadpool,514.569956


# Function to convert JSON to strings

In [8]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
       L.append(i['name'])
    return L

movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

In [9]:
movies.isnull().sum()

id              0
genres          0
keywords        0
overview        3
runtime         2
release_date    1
title_x         0
cast            0
crew            0
dtype: int64

In [10]:
movies.dropna(inplace=True)

In [11]:
def topfour(obj):
    count =1
    li=[]
    for i in ast.literal_eval(obj):
        if(count>4):
         break
        li.append(i['name'])
        count+=1
    return li

movies['cast']=movies['cast'].apply(topfour)


def fetchDirector(obj):
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
         return i['name']

movies['crew']=movies['crew'].apply(fetchDirector)

In [12]:
movies_df=movies
movies_df.head(3)

,id,genres,keywords,overview,runtime,release_date,title_x,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",162.0,2009-12-10,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",169.0,2007-05-19,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,148.0,2015-10-26,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes


# release date manipulation

In [13]:
def divisionReleaseTime(given_date):
    if(given_date<'1920'): return '1920s'
    elif(given_date>='1920' and given_date<'1930'): return '1920s'
    elif(given_date>='1930' and given_date<'1940'): return '1930s'
    elif(given_date>='1940' and given_date<'1950'): return '1940s'
    elif(given_date>='1950' and given_date<'1960'): return '1950s'
    elif(given_date>='1960' and given_date<'1970'): return '1960s'
    elif(given_date>='1970' and given_date<'1980'): return '1970s'
    elif(given_date>='1980' and given_date<'1990'): return '1980s'
    elif(given_date>='1990' and given_date<'2000'): return '1990s'
    elif(given_date>='2000' and given_date<'2010'): return '2000s'
    else: return '2010s'

movies_df['released_era']=movies_df['release_date'].apply(lambda x:divisionReleaseTime(x))

In [14]:
import requests
def fetchPosterPath(movie_id):
   result=requests.get('https://api.themoviedb.org/3/movie/{}?api_key=d39f5b74dae5e58e6b3b5542af400e19&language=en-US'.format(movie_id))
   try:
      result=result.json()
      return result['poster_path']
   except (IndexError, KeyError, TypeError):
      return "NULL"
   
movies_df['posterpath']=movies_df['id'].apply(lambda x:fetchPosterPath(x))
movies_df.head(3)

,id,genres,keywords,overview,runtime,release_date,title_x,cast,crew,released_era,posterpath
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",162.0,2009-12-10,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,2000s,/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",169.0,2007-05-19,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski,2000s,/2YMnBRh8F6fDGCCEIPk9Hb0cEyB.jpg
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,148.0,2015-10-26,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes,2010s,/zj8ongFhtWNsVlfjOGo8pSr7PQg.jpg


In [15]:
movies_df.isnull().sum()

id               0
genres           0
keywords         0
overview         0
runtime          0
release_date     0
title_x          0
cast             0
crew            29
released_era     0
posterpath      33
dtype: int64

In [16]:
movies_df.dropna(inplace=True)

# Day 4 - released era, duration manipulation


In [17]:
releasedEraList=[]

for releasedEra in movies_df['released_era']:
        if releasedEra not in releasedEraList:
            releasedEraList.append(releasedEra)

def releasedBinary_rep(release):
    releasedEraVector=[]

    for i in releasedEraList:
       if i in release:
           releasedEraVector.append(1)
       else: releasedEraVector.append(0)

    return releasedEraVector


movies_df['released_bin']=movies_df['released_era'].apply(lambda x:releasedBinary_rep(x))

In [18]:
durationList=[]
durationList.append("short")
durationList.append("medium")
durationList.append("long")
durationList

def durationBinary_rep(time):
    durationVector=[]

    for i in durationList:
        if (time<=90 and i=='short'):
           durationVector.append(1)
        elif((time>90 and time<=180) and i=='medium'):
           durationVector.append(1)
        elif(time>180 and i=='long'):
            durationVector.append(1)
        else: durationVector.append(0)

    return durationVector


movies_df['runtime_bin']=movies_df['runtime'].apply(lambda x:durationBinary_rep(x))

In [19]:
movies_df.head(2)

,id,genres,keywords,overview,runtime,release_date,title_x,cast,crew,released_era,posterpath,released_bin,runtime_bin
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",162.0,2009-12-10,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,2000s,/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0]"
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",169.0,2007-05-19,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski,2000s,/2YMnBRh8F6fDGCCEIPk9Hb0cEyB.jpg,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0]"


In [20]:
movies_df.isnull().sum()

id              0
genres          0
keywords        0
overview        0
runtime         0
release_date    0
title_x         0
cast            0
crew            0
released_era    0
posterpath      0
released_bin    0
runtime_bin     0
dtype: int64

# After removing all invalid movies

In [21]:
directorList=[]
for i in movies_df['crew']:
    if i not in directorList:
        directorList.append(i)


def binary_rep(director):
    director_rep=[]
    for directors in directorList:
        if (director==directors):
            director_rep.append(1)
        else : director_rep.append(0)
    return director_rep

movies_df['director_bin']=movies_df['crew'].apply(binary_rep)

In [22]:
castList=[]
for actor in movies_df['cast']:
    for i in actor:
        if i not in castList:
            castList.append(i)
castList

def castBinaryRep(actor):
    binaryList = []
    
    for i in castList:
        if i in actor:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList

movies_df['cast_bin']=movies_df['cast'].apply(lambda x: castBinaryRep(x))

In [23]:
genreList=[]
for genre in movies_df['genres']:
      for i in genre:
        if i not in genreList:
            genreList.append(i)
genreList

def genreBinary_rep(genres):
    genre_rep=[]
    for i in genreList:
        if i in genres:
            genre_rep.append(1)
        else : genre_rep.append(0)
    return genre_rep

movies_df['genre_bin']=movies_df['genres'].apply(lambda x:genreBinary_rep(x))


In [24]:
keywordsList=[]

for keyword in movies_df['keywords']:
    for i in keyword:
        if i not in keywordsList:
            keywordsList.append(i)

keywordsList

def keywordBinary_rep(keywords):
    keywordVector=[]

    for i in keywordsList:
       if i in keywords:
           keywordVector.append(1)
       else: keywordVector.append(0)

    return keywordVector


movies_df['keyword_bin']=movies_df['keywords'].apply(lambda x:keywordBinary_rep(x))

In [25]:
userGenreList=[]
for i in genreList:
    userGenreList.append(0)

userKeywordList=[]
for i in keywordsList:
    userKeywordList.append(0)

userDirectorList=[]
for i in directorList:
    userDirectorList.append(0)

userCastList=[]
for i in castList:
    userCastList.append(0)

userRuntimeList=[]
for i in durationList:
    userRuntimeList.append(0)

userReleasedEraList=[]
for i in releasedEraList:
    userReleasedEraList.append(0)


data = {'userGenreVector':[userGenreList],
        'userKeywordVector':[userKeywordList],
        'userCastVector':[userCastList],
        'userReleasedEraVector':[userReleasedEraList],
        'userDirectorVector':[userDirectorList],
        'userRuntimeVector':[userRuntimeList]}

userVector=pd.DataFrame(data)

In [26]:
userVector.head(1)

,userGenreVector,userKeywordVector,userCastVector,userReleasedEraVector,userDirectorVector,userRuntimeVector
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]"


In [29]:
movies_df=movies_df[['id','genres','title_x','cast','posterpath','released_bin','director_bin','cast_bin','genre_bin','keyword_bin','runtime_bin']]

In [30]:
import pickle
pickle.dump(movies_df,open('finalMovies.pkl','wb'))
pickle.dump(genreList,open('genreList.pkl','wb'))
pickle.dump(userVector,open('userDF.pkl','wb'))
pickle.dump(popular_movies_df,open('popularMovies.pkl','wb'))

# The End